In [ ]:
# Data Preprocessing, Exploratory Analysis, and Visualization
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
pd.set_option('display.max_columns', None)
pd.set_option("display.max_rows", None)

# Machine Learning
## Feature Scaling
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

## Feature Undersampling
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

## ML Models Diffrent Algorithms
import catboost as cat
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier, Pool, cv
from catboost.utils import get_roc_curve
from sklearn.model_selection import GridSearchCV


## Comparision of Performance of all Algorithms
from sklearn.metrics import roc_auc_score, roc_curve, auc
from collections import Counter

## Metrics
from sklearn.metrics import classification_report, plot_confusion_matrix

# This enables catboost to show the plot of its training process
!jupyter nbextension enable --py widgetsnbextension

# Importing Datasets

In [ ]:
#importing datasets for year-long period
fnb = pd.read_csv('AP FnB_20201101_20211031.csv')
merch = pd.read_csv('AP Merch_20201101_20211031.csv')
visit = pd.read_csv('AP Visitation_20201101_20211031.csv')
survey_QG = pd.read_csv('AP survey QG_20201101_20211031.csv')
# expire = pd.read_csv('AP Expired_20211101_20220131.csv')

# fnb Dataset

In [ ]:
fnb.info()

In [ ]:
#creating date formatted column
fnb['Date'] = fnb['year'].astype(str) + fnb['month'].astype(str)
fnb['Date'] = pd.to_datetime(fnb['Date'], format = '%Y%m')
fnb.info()

In [ ]:
fnb.head()

In [ ]:
fnb.drop(['year','month'], axis=1, inplace=True)
fnb.info()

In [ ]:
fnb_grouped=fnb.groupby(["source_id",'Date']).sum()

In [ ]:
fnb_grouped.reset_index(inplace=True)

In [ ]:
fnb_grouped.head()

# merch Dataset

In [ ]:
merch.info()

In [ ]:
#creating date formatted column
merch['Date'] = merch['year'].astype(str) + merch['month'].astype(str)
merch['Date'] = pd.to_datetime(merch['Date'], format = '%Y%m')
merch.info()

In [ ]:
merch.drop(['year','month'], axis=1, inplace=True)
merch.info()

In [ ]:
merch_grouped=merch.groupby(["source_id",'Date']).sum()

In [ ]:
merch_grouped.reset_index(inplace=True)

In [ ]:
merch_grouped.head()

In [ ]:
fnb_grouped.rename(columns={'ORDER_AMT':'Order_Amt_fnb', 'ORDER_ITEM_CNT':'Order_Item_Cnt_fnb'}, inplace=True)
merch_grouped.rename(columns={'ORDER_AMT':'Order_Amt_merch', 'ORDER_ITEM_CNT':'Order_Item_Cnt_merch'}, inplace=True)

# Merge merch and fnb datasets

In [ ]:
merged = pd.merge(fnb_grouped, merch_grouped, how="left", left_on=['source_id', 'Date'], 
                  right_on=['source_id', 'Date']);

In [ ]:
merged.head()

# visit Dataset

In [ ]:
visit['is_renewal'].value_counts()

In [ ]:
visit.head()

In [ ]:
#creating date formatted column
visit['Date'] = visit['year'].astype(str) + visit['month'].astype(str)
visit['Date'] = pd.to_datetime(visit['Date'], format = '%Y%m')
visit.info()

In [ ]:
visit.head()

In [ ]:
# Create season columns, we define Spring as month 3-5, Summer as 6-8, Autumn as 9-11, Winter as 12-2
seasons = {1:'Winter', 2:'Winter', 3:'Spring', 4:'Spring', 5:'Spring', 6:'Summer', 7:'Summer',
          8:'Summer', 9:'Autumn', 10:'Autumn', 11:'Autumn', 12:'Winter'}

In [ ]:
visit['Season'] = visit['month'].apply(lambda x: seasons[x])

In [ ]:
#season_dum = pd.get_dummies(visit['Season'])
#visit = visit.join(season_dum)
#visit.head()

In [ ]:
visit.drop(['year','month'], axis=1, inplace=True)
visit.info()

# Merge merch, fnb and visit Datasets

In [ ]:
final_merged = pd.merge(visit, merged, how="left", left_on=['source_id', 'Date'], 
                  right_on=['source_id', 'Date']);

In [ ]:
final_merged.drop(['Date'],axis=1,inplace=True)

In [ ]:
final_merged.head()

In [ ]:
# aggregate rows and get one row for each source_id
temp_fin = final_merged.drop(['Passtype','Age','MerchSpendLevel','Season'],axis=1)
temp_fin.head()

In [ ]:
temp_fin_grouped=temp_fin.groupby(["source_id"]).sum()
temp_fin_grouped.reset_index(inplace=True)

In [ ]:
# Calculate total spending and total purchased item counts for each source_id
temp_fin_grouped['Total_Order_Amt'] = temp_fin_grouped['Order_Amt_fnb'] + temp_fin_grouped['Order_Amt_merch']
temp_fin_grouped['Total_Order_Cnt'] = temp_fin_grouped['Order_Item_Cnt_fnb'] + temp_fin_grouped['Order_Item_Cnt_merch']
temp_fin_grouped.head()

In [ ]:
temp_fin_grouped.info()

In [ ]:
temp_fin_grouped['Total_Order_Amt'].describe()

In [ ]:
# get customer profile for further combination
cus_profile = final_merged[['source_id','Passtype','Age','MerchSpendLevel','Season']].copy().drop_duplicates()

In [ ]:
cus_profile['MerchSpendLevel'].value_counts()

In [ ]:
#cus_profile_dict = cus_profile.set_index('source_id').T.to_dict('list')

# Add cus profile to the merge

In [ ]:
fin_res=pd.merge(cus_profile, temp_fin_grouped, how="left", left_on=['source_id'], 
                  right_on=['source_id']);

In [ ]:
# encoding passtype, merchspendlevel and age columns
# For passtype: 0=Crystal, 1=Diamond, 2=Gold, 3=Silver
# For MerchSpendLevel: 0 = 0, 1 = <1K, 2 = >100K, 3 = >10K, 4 = >1K, 5 = >3K, 6 = >5K
# For Age: 0=Adult, 1=Select
# For Season: 0=Autumn, 1=Spring, 2=Summer, 3=Winter
fin_res["Passtype_encode"] = fin_res["Passtype"].astype('category').cat.codes
fin_res["MerchSpendLevel_encode"] = fin_res["MerchSpendLevel"].astype('category').cat.codes
fin_res['Age_encode'] = fin_res["Age"].astype('category').cat.codes
fin_res["Season_encode"] = fin_res["Season"].astype('category').cat.codes

In [ ]:
fin_res.head()

In [ ]:
fin_res.drop(['Passtype', 'MerchSpendLevel','Age','Season'],axis=1,inplace=True)
fin_res.head()

In [ ]:
fin_res.info()

In [ ]:
# Further cleaning
fin_res['is_renewal'] = fin_res['is_renewal'].apply(lambda x: 1 if x > 0 else 0)

For several source_ids, they visited the resort but didn't purchase anything for some dates. That's the reason for the 0.00 in the above datasets.

In [ ]:
# number of unique merchandise
#merch['item_desc_secondary'].str.strip()
#merch['item_desc_secondary'].nunique()

In [ ]:
#fin_res.to_csv("Preprocess_NonSurvey.csv")

# Survey QG Dataset

## Drop useless columns

In [ ]:
drop_colns = ['respid','buytime_current','LastVisit','exp_date','survey_org','famap_org_1','famap_org_2',
             'famap_org_3','famap_org_4','famap_org_5','famap_org_6','famap_org_4_other',
             'gender_org','kid_org','Kids_org','Kidsage_org_1','Kidsage_org_2','Kidsage_org_3',
              'Kidsage_org_4','Kidsage_org_5','Kidsgender_org_1','Kidsgender_org_2',
              'Kidsgender_org_3','Kidsgender_org_4','Kidsgender_org_5','Kidsage_5','Kidsgender_5',
             'kid5age','kid5gender','aprecord','aptype','aptype_label','nps_prog_mobile_1',
             'rrcry','MainlandChinaYN_org','Province_org','City_Dist_org','CountryMarket_org',
              'Pudong_org','MainlandChinaYN_Mobile','Province_Mobile','City_Dist_Mobile','CountryMarket_Mobile',
              'Pudong_Mobile','SHDLTrans_98_other','famapno_org','CAWI_Date','responseid','hhincome_org','Kids_m',
             'famap_1','famap_2','famap_3','famap_5','famap_6','famap_4','Province','City_Dist','CountryMarket',
              'Pudong']

In [ ]:
survey_QG.drop(drop_colns,axis=1,inplace=True)
survey_QG.head()

## Encode categorical columns

In [ ]:
# check object columns
# besides aptype_current, other columns are just for references and will not be used in modeling
obj_QG = survey_QG.select_dtypes(include=['object']).copy()
obj_QG.head()

In [ ]:
# aptype_current
temp_lst_QG1 = survey_QG['aptype_current'].to_list()

In [ ]:
# For current passtype: 0=Crystal, 1=Diamond, 2=Gold, 3=Silver
temp_lst_QG2 = []
for ap in temp_lst_QG1:
    if ap == '梦幻水晶卡':
        temp_lst_QG2.append(0)
    elif ap == '无限钻石卡':
        temp_lst_QG2.append(1)
    elif ap == '奇妙金卡':
        temp_lst_QG2.append(2)
    else:
        temp_lst_QG2.append(3)

In [ ]:
survey_QG['aptype_current_encode'] = pd.DataFrame(temp_lst_QG2)

## Deal with Missing Values

In [ ]:
# Create a new dataframe for modeling and drop remaining object columns
QG_modeling = survey_QG.drop(obj_QG.columns.to_list(),axis=1).copy()
QG_modeling.info()

In [ ]:
# columns with missing values
QG_modeling.columns[QG_modeling.isna().any()]

In [ ]:
#deal with missing value
QG_modeling.fillna({'Benefit_FB': QG_modeling['Benefit_FB'].mean(),
                 'Benefit_Merch': QG_modeling['Benefit_Merch'].mean(),
                 'Benefit_CAKE': QG_modeling['Benefit_CAKE'].mean(),
                 'Benefit_Stroller': QG_modeling['Benefit_Stroller'].mean(),
                 'Benefit_parking': QG_modeling['Benefit_parking'].mean(),
                 'roverall_prog': QG_modeling['roverall_prog'].mean(),
                 'adstateoe_prem': QG_modeling['adstateoe_prem'].mean(),
                 'adstateoe_entry': QG_modeling['adstateoe_entry'].mean(),
                 'adstateoe_event': QG_modeling['adstateoe_event'].mean(),
                 'ovvalue_ap': QG_modeling['ovvalue_ap'].mean(),
                 'nps_prog_1': QG_modeling['nps_prog_1'].mean(),
                 'rtintent_AP': QG_modeling['rtintent_AP'].mean(),
                 'adstatecry_rdate': QG_modeling['adstatecry_rdate'].mean(),
                 'adstatecry_window': QG_modeling['adstatecry_window'].mean(),
                 'adstatecry_cancel': QG_modeling['adstatecry_cancel'].mean(),
                 'adstatecry_punish': QG_modeling['adstatecry_punish'].mean(),
                 'adstatecry_rprocess': QG_modeling['adstatecry_rprocess'].mean(),
                 'adstatecry_entry': QG_modeling['adstatecry_entry'].mean(),
                 'adstatecry_hc': QG_modeling['adstatecry_hc'].mean(),
                 'return_nonv': 6,
                 'rcoupon_CAKE': QG_modeling['rcoupon_CAKE'].mean(),
                 'rcoupon_FB': QG_modeling['rcoupon_FB'].mean(),
                 'rcoupon_Merch': QG_modeling['rcoupon_Merch'].mean(),
                 'rcoupon_parking': QG_modeling['rcoupon_parking'].mean(),
                 'rcoupon_Stroller': QG_modeling['rcoupon_Stroller'].mean(),
                 }, inplace=True)

In [ ]:
# Fill rest of the missing values wiith 0
QG_modeling.fillna(0, inplace=True)

In [ ]:
# Double Check if there is any missing value remains in the dataset
# QG_modeling.columns[QG_modeling.isna().any()]
msno.matrix(QG_modeling)

In [ ]:
QG_modeling.head()

## Notes

In [ ]:
survey_QG.loc[survey_QG['source_id']==256066]

In [ ]:
fin_res.loc[fin_res['source_id']==256066]

- Since it is a quaterly survey, some AP holders took the survey multiple times.

# Combining QG_modeling with fin_res Datasets

In [ ]:
#Final DataFrame used for modeling
df=pd.merge(fin_res, QG_modeling, how="left", left_on=['source_id'], 
                  right_on=['source_id']);

## Dealing with missing values

In [ ]:
df.fillna(0,inplace=True)

## Further Feature Engineering

In [ ]:
#Interactive Column Creation
df['SpendPerVisit']=df['Total_Order_Amt']/df['visit_times']

In [ ]:
#Can decide to drop additional columns that may affect output of models
drop_colns_df = ['visit_times','Order_Amt_fnb','Order_Item_Cnt_fnb','Order_Amt_merch','Order_Item_Cnt_merch',
                 'Total_Order_Amt','Total_Order_Cnt','TotalMerchSpend','aptype_current_encode','ap_stage',
                 'TotalFnBSpend','Totalspend','Spendpervisit','commrec_98','Kidsgender_2','commrec_5','commrec_4',
                 'kid2age','kid3age','kid3gender','kid4age','adstateoe_entry','Kidsage_4','Kidsage_3','vbarrier_3',
                 'vbarrier_5','Kidsgender_3','vbarrier_9','vbarrier_10','vbarrier_98','commrec_1','vbarrier_6',
                'Kidsgender_4','kid4gender','renew_to_NY']
df.drop(drop_colns_df,axis=1,inplace=True)

In [ ]:
len(df)

In [ ]:
df.head()

In [ ]:
# Check if there is an imbalance in the classes present in the target variable ('is_renewal')
renewal_counts = df["is_renewal"].value_counts()
temp_df = pd.DataFrame({
    "Yes": renewal_counts.index,
    "No": renewal_counts.values
})
 
plt.figure(figsize = (18,8))
sns.barplot(x = "Yes", y = "No", data = temp_df)
plt.xticks(rotation=90)
plt.show()

In [ ]:
#Slightly imbalanced dataset
df.is_renewal.value_counts()

# Outlier Detection - Optional

In [ ]:
#Outlier detection and information for each column

#def find_outliers_IQR(df):
    #q1=df.quantile(0.25)

    #q3=df.quantile(0.75)

    #IQR=q3-q1

    #outliers = df[((df<(q1-1.5*IQR)) | (df>(q3+1.5*IQR)))]

    #return outliers

#for column in df:
    # outliers = find_outliers_IQR(df[column])

    #print(df[column].name,':',"number of outliers: "+ str(len(outliers)))

    #print(df[column].name,':',"max outlier value: "+ str(outliers.max()))

    #print(df[column].name,':',"min outlier value: "+ str(outliers.min()))
    
    

In [ ]:
#Removing Outliers - feel free to change the percentile threshold (40/60 -> 15/85, etc.)

#for column in df:
    #Q1 = np.percentile(df[column], 40,
                   #interpolation = 'midpoint')
    #Q3 = np.percentile(df[column], 60,
                   #interpolation = 'midpoint')
    #IQR = Q3 - Q1
 
    #print("Old Shape: ", df.shape)
 
    # Upper bound
    #upper = np.where(df[column] >= (Q3+1.5*IQR))
    # Lower bound
    #lower = np.where(df[column] <= (Q1-1.5*IQR))
 
    #''' Removing the Outliers '''
    #df.drop(upper[0], inplace = True)
    #df.drop(lower[0], inplace = True)
 
    #print("New Shape: ", df.shape)

## Data Visualization

In [ ]:
# Histograms of all features in the final dataframe - want to keep an eye out for values that wouldn't make sense (ie. negatives)
df.hist(figsize=(20,20))
plt.show()

In [ ]:
# Correlation Matrix Expcet For 'source_id' and 'is_renewal - keep track of which features have high/low correlation
corr = df.drop(['source_id','is_renewal'],axis=1).corr()
mask = np.zeros_like(corr)
mask[np.triu_indices_from(mask)] = True
# Heatmap
plt.figure(figsize=(15, 10))
sns.heatmap(corr,
            vmax=.5,
            mask=mask,
            # annot=True, fmt='.2f',
            linewidths=.2, cmap="YlGnBu")

In [ ]:
def get_redundant_pairs(df):
    '''Get diagonal and lower triangular pairs of correlation matrix'''
    pairs_to_drop = set()
    cols = df.columns
    for i in range(0, df.shape[1]):
        for j in range(0, i+1):
            pairs_to_drop.add((cols[i], cols[j]))
    return pairs_to_drop

def get_top_abs_correlations(df, n=5):
    au_corr = df.corr().abs().unstack()
    labels_to_drop = get_redundant_pairs(df)
    au_corr = au_corr.drop(labels=labels_to_drop).sort_values(ascending=False)
    return au_corr[0:n]

print("Top Absolute Correlations")
print(get_top_abs_correlations(df.drop(['source_id','is_renewal'],axis=1), 5))

Additional Feature Engineering Ideas:
1) Additional Interactive Variables if dependence between variables can be captured
2) Binning of Renewal Date periods if possible (seperate into Monthly increments)
3) Feature Selection (What to drop and what to keep based on correlation to each other and label specifically)

# Modeling

In [ ]:
X = df.drop('is_renewal',axis=1)
y = df.is_renewal

In [ ]:
X.head()
# For MerchSpendLevel: 0 = 0, 1 = <1K, 2 = >100K, 3 = >10K, 4 = >1K, 5 = >3K, 6 = >5K

In [ ]:
# Split training set inyo two sets to build and validate the model 
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [ ]:
#assigning source_id values for predictions at the end
x_test_id=pd.DataFrame(x_test['source_id'])
x_test_id.head()

In [ ]:
x_test.head()

In [ ]:
x_test.drop(['source_id'],axis=1,inplace=True)
x_train.drop(['source_id'],axis=1,inplace=True)

## Feature Undersampling

In [ ]:
#Balancing classes 
under_sampler = RandomUnderSampler(random_state=42)
x_train, y_train = under_sampler.fit_resample(x_train, y_train)
print(f"Training target statistics: {Counter(y_train)}")
print(f"Testing target statistics: {Counter(y_test)}")

## Feature Scaling 

In [ ]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_train

In [ ]:
scaler = StandardScaler()
x_test = scaler.fit_transform(x_test)
x_test

## Catboost

In [ ]:
#Grid Search to find best hyperparameters for the CatBoost model - can change values in lists below
catb_para = {'learning_rate': [0.01, 0.03, 0.05, 0.07, 0.1,0.13,0.15,0.2],
        'depth': [4, 6, 8, 10,12,14,16,20],
        'iterations':[10, 50, 100, 200, 400,600,800,1000]}

catb = GridSearchCV(estimator=CatBoostClassifier(), param_grid=catb_para, cv=5, scoring='accuracy',
                      verbose=1,n_jobs = -1)

In [ ]:
catb.fit(x_train, y_train)

In [ ]:
#Best hyperparameters based on GridSearch
print(catb.best_params_)

In [ ]:
# catboost_pool = cat.Pool(x_train, y_train)

In [ ]:
pd.DataFrame({'feature_importance': catb.best_estimator_.feature_importances_, 
              'feature_names': X.drop(['source_id'],axis=1).columns}).sort_values(by=['feature_importance'], 
                                                           ascending=False)

In [ ]:
print(classification_report(y_test, catb.predict(x_test)))
plot_confusion_matrix(catb, x_test, y_test)

# MLP

In [ ]:
#Grid Search to find best hyperparameters for the MLP model - can change values in lists below
MLP_para = {'solver': ['lbfgs'], 'max_iter': [1000,1300,1500,1800,2000], 'alpha': [1e-3,1e-4,1e-5], 
            'hidden_layer_sizes':[(4,2),(5,2),(6,3)], 'random_state':[0,1,2]}
MLP = GridSearchCV(MLPClassifier(), MLP_para, n_jobs=-1, verbose=1, cv=5, scoring='accuracy')

In [ ]:
# MLP = MLPClassifier(solver='lbfgs', alpha=1e-5,
#                      hidden_layer_sizes=(5, 2), random_state=1)

In [ ]:
MLP.fit(x_train, y_train)

In [ ]:
#Best hyperparameters based on GridSearch
print(MLP.best_params_)

In [ ]:
print(classification_report(y_test,MLP.predict(x_test)))
plot_confusion_matrix(MLP, x_test, y_test)

# Decision Tree

In [ ]:
#Grid Search to find best hyperparameters for the Decision Tree model - can change values in lists below
tree_para = {'criterion':['gini','entropy'],'max_depth':[4,6,8,10,20,40,70,120,150],'min_samples_split':[4,5,6,7],
            'random_state':[0,1,2]}
tree = GridSearchCV(DecisionTreeClassifier(), tree_para, n_jobs=-1, verbose=1, cv=5, scoring='accuracy')

In [ ]:
tree.fit(x_train, y_train)

In [ ]:
#Best hyperparameters based on GridSearch
print(tree.best_params_)

In [ ]:
pd.DataFrame({'feature_importance': tree.best_estimator_.feature_importances_, 
              'feature_names': X.drop(['source_id'],axis=1).columns}).sort_values(by=['feature_importance'], 
                                                           ascending=False)

In [ ]:
#tree = DecisionTreeClassifier(random_state=0, max_depth=6, min_samples_split=15)
print(classification_report(y_test, tree.predict(x_test)))
plot_confusion_matrix(tree, x_test, y_test)

# Random Forest

- Best Combo from grid_search: RandomForestClassifier(max_depth=110, max_features=3, min_samples_leaf=3,
                       min_samples_split=8, n_estimators=300)
- The performance is worse wiith the best combo, why?

In [ ]:
#Grid Search to find best hyperparameters for the Random Forest model - can change values in lists below
forest_para = {'criterion':['gini','entropy'],'max_depth':[110,115,116,117,118,119,120], 'max_features':[1,2,3,4,5],
               'n_estimators':[250,300,350]}
forest = GridSearchCV(estimator=RandomForestClassifier(), param_grid=forest_para, cv=5, scoring='accuracy',
                      verbose=1,n_jobs = -1)

In [ ]:
forest.fit(x_train, y_train)

In [ ]:
#Best hyperparameters based on GridSearch
print(forest.best_params_)

In [ ]:
pd.DataFrame({'feature_importance': forest.best_estimator_.feature_importances_, 
              'feature_names': X.drop(['source_id'],axis=1).columns}).sort_values(by=['feature_importance'], 
                                                           ascending=False)

In [ ]:
print(classification_report(y_test, forest.predict(x_test)))
plot_confusion_matrix(forest, x_test, y_test)

# Gradient Boosting

In [ ]:
#Grid Search to find best hyperparameters for the Gradient Boosting model - can change values in lists below
gboost_para = {
    "learning_rate": [0.01, 0.05, 0.1, 0.2],
    "max_depth":[3,5,8],
    "n_estimators":[10,100,110,200]
    }

gboost = GridSearchCV(estimator=GradientBoostingClassifier(), param_grid=gboost_para, cv=5, scoring='accuracy',
                      verbose=1, n_jobs = -1)

In [ ]:
# gboost = GradientBoostingClassifier(n_estimators=200, max_depth=2, random_state=0)
gboost.fit(x_train, y_train)

In [ ]:
#Best hyperparameters based on GridSearch
print(gboost.best_params_)

In [ ]:
pd.DataFrame({'feature_importance': gboost.best_estimator_.feature_importances_, 
              'feature_names': X.drop(['source_id'],axis=1).columns}).sort_values(by=['feature_importance'], 
                                                           ascending=False)

In [ ]:
print(classification_report(y_test, gboost.predict(x_test)))
plot_confusion_matrix(gboost, x_test, y_test) 

# KNN

In [ ]:
#Grid Search to find best hyperparameters for the KNN model - can change values in lists below
k_range = list(range(1, 31))
weights=['distance']
knn_para = dict(n_neighbors=k_range, weights=weights)
  
KNN = GridSearchCV(KNeighborsClassifier(), knn_para, cv=5, scoring='accuracy', verbose=1)

In [ ]:
#KNN = KNeighborsClassifier(n_neighbors=5, weights='distance')
# fitting the model for grid search
KNN.fit(x_train, y_train)

In [ ]:
#Best hyperparameters based on GridSearch
print(KNN.best_params_)

In [ ]:
print(classification_report(y_test, KNN.predict(x_test)))
plot_confusion_matrix(KNN, x_test, y_test) 

# Stacking

In [ ]:
#Ensemble machine learning algorithm to combine results from our other models
train_model1=tree.predict(x_test)
train_model2=forest.predict(x_test)
train_model3=gboost.predict(x_test)
train_model4=KNN.predict(x_test)
train_model5=catb.predict(x_test)
train_model6=MLP.predict(x_test)

In [ ]:
stacked_train=np.column_stack((train_model1,train_model2,train_model3,train_model4,train_model5,train_model6))

In [ ]:
meta_model=LogisticRegression()

In [ ]:
meta_model.fit(stacked_train,y_test)

In [ ]:
model1_pred=tree.predict(x_test)
model2_pred=forest.predict(x_test)
model3_pred=gboost.predict(x_test)
model4_pred=KNN.predict(x_test)
model5_pred=catb.predict(x_test)
model6_pred=MLP.predict(x_test)

In [ ]:
stacked_pred=np.column_stack((model1_pred,model2_pred,model3_pred,model4_pred,model5_pred,model6_pred))

In [ ]:
print(classification_report(y_test, meta_model.predict(stacked_pred)))
plot_confusion_matrix(meta_model, stacked_pred, y_test) 

## AUC&ROC - Evaluating our results

In [ ]:
p1 = tree.predict_proba(x_test)
p2 = forest.predict_proba(x_test)
p3 = gboost.predict_proba(x_test)
p4 = KNN.predict_proba(x_test)
p5 = catb.predict_proba(x_test)
p6 = MLP.predict_proba(x_test)

In [ ]:
# auc scores to evaluate our modeling results -> (65-80% probability of distinguishing a "Non-Renewer" from a "Renewer")
auc_score1 = roc_auc_score(y_test, p1[:,1])
auc_score2 = roc_auc_score(y_test, p2[:,1])
auc_score3 = roc_auc_score(y_test, p3[:,1])
auc_score4 = roc_auc_score(y_test, p4[:,1])
auc_score5 = roc_auc_score(y_test, p5[:,1])
auc_score6 = roc_auc_score(y_test, p6[:,1])

print(auc_score1, auc_score2, auc_score3, auc_score4, auc_score5, auc_score6)

In [ ]:
# roc curve for models
fpr1, tpr1, thresh1 = roc_curve(y_test, p1[:,1], pos_label=1)
fpr2, tpr2, thresh2 = roc_curve(y_test, p2[:,1], pos_label=1)
fpr3, tpr3, thresh3 = roc_curve(y_test, p3[:,1], pos_label=1)
fpr4, tpr4, thresh4 = roc_curve(y_test, p4[:,1], pos_label=1)
fpr5, tpr5, thresh5 = roc_curve(y_test, p5[:,1], pos_label=1)
fpr6, tpr6, thresh6 = roc_curve(y_test, p6[:,1], pos_label=1)
fpr_final, tpr_final, thresh7 = roc_curve(y_test, meta_model.predict(stacked_pred))
roc_auc_final = auc(fpr_final, tpr_final)

# roc curve for tpr = fpr 
random_probs = [0 for i in range(len(y_test))]
p_fpr, p_tpr, _ = roc_curve(y_test, random_probs, pos_label=1)

In [ ]:
plt.plot(fpr1, tpr1, linestyle='--', label='Decision Tree (area = %0.2f)' % auc_score1)
plt.plot(fpr2, tpr2, linestyle='--', label='Random Forest (area = %0.2f)' % auc_score2)
plt.plot(fpr3, tpr3, linestyle='--', label='Gradient Boost (area = %0.2f)' % auc_score3)
plt.plot(fpr4, tpr4, linestyle='--', label='KNN (area = %0.2f)' % auc_score4)
plt.plot(fpr5, tpr5, linestyle='--', label='CatBoost (area = %0.2f)' % auc_score5)
plt.plot(fpr6, tpr6, linestyle='--', label='MLP (area = %0.2f)' % auc_score6)
plt.plot(fpr_final, tpr_final, linestyle="--", label="Stacking (area = %0.2f)" % roc_auc_final)
plt.plot(p_fpr, p_tpr, color='black', lw=2)
plt.title('ROC Curves')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive rate')
plt.legend(loc='lower right')

# MAPE for models - Another evaluation metric

In [ ]:
# from sklearn.metrics import mean_absolute_percentage_error

In [ ]:
def MAPE(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / np.maximum(np.ones(len(y_true)), np.abs(y_true))))*100

In [ ]:
MAPE(catb.predict(x_test),y_test)

In [ ]:
MAPE(MLP.predict(x_test),y_test)

In [ ]:
MAPE(KNN.predict(x_test),y_test)

In [ ]:
MAPE(gboost.predict(x_test),y_test)

In [ ]:
MAPE(forest.predict(x_test),y_test)

In [ ]:
MAPE(tree.predict(x_test),y_test)

In [ ]:
MAPE(meta_model.predict(stacked_pred),y_test)

# Next (Optional) Steps - Making Renewal Predictions for each Customer

In [ ]:
#For general purposes, you'd have an unseen, test dataframe to predict on, but for our case, we'll the initial x_test dataset/id features

#Want to use model with highest predictive accuracy to test on final results***
x_test_id['Renewed?']=catb.predict(x_test)
x_test_id.head()


From here, you could go on to test different discount rates for each (churning) customer and see what discount threshold is needed to prevent churn. 

Example:

Evaluating the impact of an offer to test retention offering scenarios:

acceptance_rate_nonchurn = 1,

acceptance_rate_churn = 0.48,

threshold = 0.48,

base_price = $10,

discount_amount = $0.48

So what would the churn rate be at a price of $9.52?

Can change each of these parameters and see likelihood of churn/not churn

**REQUIRES ADDITIONAL CODE/DATA**
